Возьмите реализацию класса HashTable из лекционных материалов и выполните
следующие доработки:
1. Реализуйте квадратичное пробирование как технику повторного хеширования.
2. Реализуйте работу с функцией len (переопределите метод __len__).
3. Реализуйте работу оператора in (переопределите метод __contains__).
4. Переделайте метод put таким образом, чтобы таблица автоматически меняла размер,
когда загрузочный фактор становится больше значения 0.7. Размер должен
увеличиваться примерно в два раза до ближайшего подходящего простого числа.
5. Реализуйте работу оператора del (переопределите метод __delitem__) для удаления
элемента таблицы. Таблица должна автоматически менять размер, когда
загрузочный фактор становится меньше значения 0.2. Размер должен уменьшаться
примерно в два раза до ближайшего подходящего простого числа.
Все выполненные доработки должны быть протестированы.

In [ ]:
class HashTable:
    """
    Хеш-таблица с квадратичным пробированием и динамическим изменением размера
    """
    
    def __init__(self, size=11):
        """
        Инициализация хеш-таблицы
        :param size: начальный размер таблицы (простое число)
        """
        self.size = self._next_prime(size)
        self.slots = [None] * self.size
        self.data = [None] * self.size
        self.count = 0  # количество элементов в таблице
    
    def _is_prime(self, n):
        """Проверка, является ли число простым"""
        if n < 2:
            return False
        if n == 2:
            return True
        if n % 2 == 0:
            return False
        for i in range(3, int(n ** 0.5) + 1, 2):
            if n % i == 0:
                return False
        return True
    
    def _next_prime(self, n):
        """Найти ближайшее простое число >= n"""
        if n < 2:
            return 2
        prime = n
        while not self._is_prime(prime):
            prime += 1
        return prime
    
    def _hash_function(self, key):
        """Базовая хеш-функция"""
        return hash(key) % self.size
    
    def _rehash(self, old_hash, i):
        """
        Квадратичное пробирование: h(k, i) = (h(k) + i²) mod m
        :param old_hash: исходное хеш-значение
        :param i: номер попытки (начиная с 1)
        :return: новое хеш-значение
        """
        return (old_hash + i * i) % self.size
    
    def _resize(self, new_size):
        """
        Изменение размера таблицы с перехешированием всех элементов
        :param new_size: новый размер таблицы
        """
        # Сохраняем старые данные
        old_slots = self.slots
        old_data = self.data
        
        # Создаем новую таблицу
        self.size = self._next_prime(new_size)
        self.slots = [None] * self.size
        self.data = [None] * self.size
        self.count = 0
        
        # Перехешируем все элементы
        for i in range(len(old_slots)):
            if old_slots[i] is not None:
                self.put(old_slots[i], old_data[i])
    
    def put(self, key, value):
        """
        Добавление или обновление элемента в таблице
        :param key: ключ
        :param value: значение
        """
        # Проверка загрузочного фактора перед вставкой
        load_factor = self.count / self.size
        if load_factor > 0.7:
            self._resize(self.size * 2)
        
        hash_value = self._hash_function(key)
        
        # Если слот пустой, вставляем
        if self.slots[hash_value] is None:
            self.slots[hash_value] = key
            self.data[hash_value] = value
            self.count += 1
        else:
            # Если ключ совпадает, обновляем значение
            if self.slots[hash_value] == key:
                self.data[hash_value] = value
            else:
                # Квадратичное пробирование
                i = 1
                next_slot = self._rehash(hash_value, i)
                
                while self.slots[next_slot] is not None and self.slots[next_slot] != key:
                    i += 1
                    next_slot = self._rehash(hash_value, i)
                    
                    # Защита от бесконечного цикла
                    if i > self.size:
                        raise Exception("Хеш-таблица переполнена")
                
                if self.slots[next_slot] is None:
                    self.slots[next_slot] = key
                    self.data[next_slot] = value
                    self.count += 1
                else:
                    # Обновление существующего ключа
                    self.data[next_slot] = value
    
    def get(self, key):
        """
        Получение значения по ключу
        :param key: ключ
        :return: значение или None
        """
        start_slot = self._hash_function(key)
        
        position = start_slot
        i = 0
        
        while self.slots[position] is not None:
            if self.slots[position] == key:
                return self.data[position]
            
            i += 1
            position = self._rehash(start_slot, i)
            
            # Если вернулись к начальной позиции или прошли всю таблицу
            if i > self.size:
                break
        
        return None
    
    def __getitem__(self, key):
        """Поддержка синтаксиса table[key]"""
        value = self.get(key)
        if value is None:
            raise KeyError(f"Ключ '{key}' не найден")
        return value
    
    def __setitem__(self, key, value):
        """Поддержка синтаксиса table[key] = value"""
        self.put(key, value)
    
    def __len__(self):
        """
        Возвращает количество элементов в таблице
        :return: количество элементов
        """
        return self.count
    
    def __contains__(self, key):
        """
        Проверка наличия ключа в таблице (оператор in)
        :param key: ключ для проверки
        :return: True если ключ существует, False иначе
        """
        start_slot = self._hash_function(key)
        
        position = start_slot
        i = 0
        
        while self.slots[position] is not None:
            if self.slots[position] == key:
                return True
            
            i += 1
            position = self._rehash(start_slot, i)
            
            if i > self.size:
                break
        
        return False
    
    def __delitem__(self, key):
        """
        Удаление элемента из таблицы (оператор del)
        :param key: ключ для удаления
        """
        start_slot = self._hash_function(key)
        
        position = start_slot
        i = 0
        found = False
        
        while self.slots[position] is not None:
            if self.slots[position] == key:
                self.slots[position] = None
                self.data[position] = None
                self.count -= 1
                found = True
                break
            
            i += 1
            position = self._rehash(start_slot, i)
            
            if i > self.size:
                break
        
        if not found:
            raise KeyError(f"Ключ '{key}' не найден")
        
        # Проверка загрузочного фактора после удаления
        if self.size > 11:  # Не уменьшаем ниже начального размера
            load_factor = self.count / self.size
            if load_factor < 0.2:
                new_size = max(11, self.size // 2)
                self._resize(new_size)
    
    def __str__(self):
        """Строковое представление таблицы"""
        items = []
        for i in range(self.size):
            if self.slots[i] is not None:
                items.append(f"{self.slots[i]}: {self.data[i]}")
        return "{" + ", ".join(items) + "}"
    
    def load_factor(self):
        """Возвращает текущий загрузочный фактор"""
        return self.count / self.size if self.size > 0 else 0
    
    def get_stats(self):
        """Возвращает статистику таблицы"""
        return {
            'size': self.size,
            'count': self.count,
            'load_factor': self.load_factor()
        }


# ===== ТЕСТИРОВАНИЕ =====

def test_basic_operations():
    """Тест базовых операций"""
    print("Тест 1: Базовые операции")
    ht = HashTable()
    
    # Вставка элементов
    ht.put('apple', 10)
    ht.put('banana', 20)
    ht.put('orange', 30)
    
    print(f"Таблица: {ht}")
    print(f"apple: {ht.get('apple')}")
    print(f"banana: {ht.get('banana')}")
    print(f"Статистика: {ht.get_stats()}")
    print("Тест пройден\n")


def test_len_operator():
    """Тест оператора len"""
    print("Тест 2: Оператор len")
    ht = HashTable()
    
    print(f"Начальная длина: {len(ht)}")
    
    ht['a'] = 1
    ht['b'] = 2
    ht['c'] = 3
    
    print(f"После добавления 3 элементов: {len(ht)}")
    assert len(ht) == 3, "Длина должна быть 3"
    print("Тест пройден\n")


def test_contains_operator():
    """Тест оператора in"""
    print("Тест 3: Оператор in")
    ht = HashTable()
    
    ht['key1'] = 'value1'
    ht['key2'] = 'value2'
    
    print(f"'key1' in ht: {'key1' in ht}")
    print(f"'key2' in ht: {'key2' in ht}")
    print(f"'key3' in ht: {'key3' in ht}")
    
    assert 'key1' in ht, "key1 должен быть в таблице"
    assert 'key3' not in ht, "key3 не должен быть в таблице"
    print("Тест пройден\n")


def test_quadratic_probing():
    """Тест квадратичного пробирования"""
    print("Тест 4: Квадратичное пробирование")
    ht = HashTable(size=7)
    
    # Добавляем элементы, которые могут вызвать коллизии
    keys = [7, 14, 21, 28, 35]  # Все дадут одинаковый хеш при size=7
    for i, key in enumerate(keys):
        ht.put(key, f'value_{i}')
    
    # Проверяем, что все элементы можно найти
    for i, key in enumerate(keys):
        assert ht.get(key) == f'value_{i}', f"Значение для ключа {key} должно быть value_{i}"
        print(f"Ключ {key}: {ht.get(key)}")
    
    print("Тест пройден (коллизии разрешены квадратичным пробированием)\n")


def test_auto_resize_up():
    """Тест автоматического увеличения размера"""
    print("Тест 5: Автоматическое увеличение размера (загрузка > 0.7)")
    ht = HashTable(size=11)
    
    print(f"Начальный размер: {ht.size}, count: {ht.count}")
    
    # Добавляем элементы до превышения загрузочного фактора 0.7
    for i in range(20):
        ht.put(f'key_{i}', f'value_{i}')
        if i in [7, 15]:
            stats = ht.get_stats()
            print(f"После {i+1} элементов: size={stats['size']}, count={stats['count']}, load={stats['load_factor']:.2f}")
    
    stats = ht.get_stats()
    print(f"Финальный размер: size={stats['size']}, count={stats['count']}, load={stats['load_factor']:.2f}")
    
    # Проверяем, что все элементы доступны
    for i in range(20):
        assert ht.get(f'key_{i}') == f'value_{i}', f"Элемент key_{i} потерян после resize"
    
    print("Тест пройден\n")


def test_del_operator():
    """Тест оператора del"""
    print("Тест 6: Оператор del")
    ht = HashTable()
    
    # Добавляем элементы
    for i in range(5):
        ht[f'key_{i}'] = f'value_{i}'
    
    print(f"Начальная длина: {len(ht)}")
    print(f"Таблица: {ht}")
    
    # Удаляем элемент
    del ht['key_2']
    print(f"После удаления 'key_2': длина={len(ht)}")
    print(f"'key_2' in ht: {'key_2' in ht}")
    
    assert len(ht) == 4, "Длина должна быть 4"
    assert 'key_2' not in ht, "key_2 не должен быть в таблице"
    
    # Проверяем, что другие элементы остались
    assert ht['key_0'] == 'value_0', "key_0 должен остаться"
    assert ht['key_4'] == 'value_4', "key_4 должен остаться"
    
    print("Тест пройден\n")


def test_auto_resize_down():
    """Тест автоматического уменьшения размера"""
    print("Тест 7: Автоматическое уменьшение размера (загрузка < 0.2)")
    ht = HashTable(size=11)
    
    # Добавляем много элементов (таблица вырастет)
    for i in range(30):
        ht[f'key_{i}'] = f'value_{i}'
    
    stats = ht.get_stats()
    print(f"После добавления 30 элементов: size={stats['size']}, count={stats['count']}, load={stats['load_factor']:.2f}")
    size_before_deletions = stats['size']
    
    # Удаляем большую часть элементов
    for i in range(25):
        del ht[f'key_{i}']
    
    stats = ht.get_stats()
    print(f"После удаления 25 элементов: size={stats['size']}, count={stats['count']}, load={stats['load_factor']:.2f}")
    
    # Проверяем, что размер уменьшился
    assert stats['size'] < size_before_deletions, "Размер таблицы должен уменьшиться"
    
    # Проверяем, что оставшиеся элементы доступны
    for i in range(25, 30):
        assert ht[f'key_{i}'] == f'value_{i}', f"Элемент key_{i} потерян после resize down"
    
    print("Тест пройден\n")


def test_update_existing_key():
    """Тест обновления существующего ключа"""
    print("Tест 8: Обновление существующего ключа")
    ht = HashTable()
    
    ht['key'] = 'value1'
    print(f"Начальное значение: {ht['key']}")
    
    ht['key'] = 'value2'
    print(f"Обновленное значение: {ht['key']}")
    
    assert len(ht) == 1, "Длина должна остаться 1"
    assert ht['key'] == 'value2', "Значение должно быть обновлено"
    
    print("Тест пройден\n")


def test_error_handling():
    """Тест обработки ошибок"""
    print("Тест 9: Обработка ошибок")
    ht = HashTable()
    
    ht['key1'] = 'value1'
    
    # Тест KeyError при получении несуществующего ключа
    try:
        value = ht['nonexistent']
        print("Должен был выброситься KeyError")
    except KeyError as e:
        print(f"KeyError при получении: {e}")
    
    # Тест KeyError при удалении несуществующего ключа
    try:
        del ht['nonexistent']
        print("Должен был выброситься KeyError")
    except KeyError as e:
        print(f"KeyError при удалении: {e}")
    
    print("Тест пройден\n")


def test_stress():
    """Стресс-тест с большим количеством операций"""
    print("Тест 10: Стресс-тест")
    ht = HashTable()
    
    n = 1000
    
    # Вставка
    for i in range(n):
        ht[i] = i * 10
    
    print(f"Вставлено {n} элементов")
    stats = ht.get_stats()
    print(f"Статистика: size={stats['size']}, count={stats['count']}, load={stats['load_factor']:.2f}")
    
    # Проверка
    for i in range(n):
        assert i in ht, f"Ключ {i} должен быть в таблице"
        assert ht[i] == i * 10, f"Значение для ключа {i} неверно"
    
    print(f"Все {n} элементов найдены корректно")
    
    # Удаление половины
    for i in range(0, n, 2):
        del ht[i]
    
    stats = ht.get_stats()
    print(f"После удаления {n//2} элементов: size={stats['size']}, count={stats['count']}, load={stats['load_factor']:.2f}")
    
    # Проверка оставшихся
    for i in range(1, n, 2):
        assert i in ht, f"Ключ {i} должен остаться в таблице"
    

# Запуск всех тестов
if __name__ == "__main__":
    print("Тестирование хэш-таблицы с квадр. пробированием")
    
    test_basic_operations()
    test_len_operator()
    test_contains_operator()
    test_quadratic_probing()
    test_auto_resize_up()
    test_del_operator()
    test_auto_resize_down()
    test_update_existing_key()
    test_error_handling()
    test_stress()
